# Minimum spanning trees
### Michiel Stock
### 2019-2020

In [ ]:
using STMO
using Plots

In this chapter, we will introduce minimization problems on graphs. We will study an elementary problem in computer science: finding the *minimum spanning tree* (MST). The minimum spanning tree has plenty of real-world applications, such as designing computer-, telecommunication- or other supply networks, computer graphics, and bioinformatics. Interestingly, there are efficient algorithms that can find the minimum spanning tree for even huge problems! Along our way, we will also be getting acquainted with some new types of data structures other than simple matrices.


# Graphs in Julia

Graphs are principal tools in computer science. Most programming languages provide interfaces for graphs; Julia is no exception. A simple package for working with is [`LightGraphs.jl`](https://github.com/JuliaGraphs/LightGraphs.jl). In this course, we will limit ourselves to using the basic data structures provided by these languages, arrays, sets, and dictionaries. The type system allows us to formally encode how we will represent graph structures.

```
EdgeList{T} = Array{Tuple{T,T},1}
WeightedEdgeList{R,T} = Array{Tuple{R,T,T},1}
Vertices{T} = Array{T,1}
AdjList{R,T} = Dict{T,Array{Tuple{R,T},1}}
```

Don't worry if you don't fully understand the above. It makes use of the type system and it allows us to use dispatch to select the best function.

Consider the following example graph:

![A small example to show how to implement graphs in Julia.](Figures/graph.png)

This graph can be represented using an *adjacency list*. We do this using a `Dict`. Every vertex is a key with the adjacent vertices given as a `set` containing tuples `(weight, neighbor)`. The weight is first because this makes it easy to compare the weights of two edges. Note that for every ingoing edge, there is also an outgoing edge, this is an undirected graph.

In [ ]:
graph = Dict(
    'A' => [(2, 'B'), (3, 'D')],
    'B' => [(2, 'A'), (1, 'C'), (2, 'E')],
    'C' => [(1, 'B'), (2, 'D'), (1, 'E')],
    'D' => [(2, 'C'), (3, 'A'), (3, 'E')],
    'E' => [(2, 'B'), (1, 'C'), (3, 'D')]
);

Sometimes we will use an *edge list*, i.e., a list of (weighted) edges. The edge list is often a more compact way of storing a graph. We show the edge list for the example graph below. Note that again every edge is double: we include an in- and outgoing edge.

In [ ]:
edges = [
 (2, 'B', 'A'),
 (3, 'D', 'A'),
 (2, 'C', 'D'),
 (3, 'A', 'D'),
 (3, 'E', 'D'),
 (2, 'B', 'E'),
 (3, 'D', 'E'),
 (1, 'C', 'E'),
 (2, 'E', 'B'),
 (2, 'A', 'B'),
 (1, 'C', 'B'),
 (1, 'E', 'C'),
 (1, 'B', 'C'),
 (2, 'D', 'C')];

We can quickly turn one representation in the other (with a time complexity proportional to the number of edges) using the provided functions `edges2adjlist` and `adjlist2edges`.

In [ ]:
adjlist2edges(graph)

In [ ]:
edges2adjlist(edges)

# Some useful data structures

## Disjoint-set data structure

Implementing an algorithm for finding the minimum spanning tree is reasonably straightforward. The only bottleneck is that the algorithm requires the disjoint-set data structure to keep track of a set partitioned in several disjoined subsets.

For example, consider the following initial set of eight elements.

![](Figures/disjointset1.png)

We decide to group elements A, B, and C together in a subset and F and G in another subset.

![](Figures/disjointset2.png)

The disjoint-set data structure supports the following operations:

- **Find**: check which subset an element is in. Is typically used to check whether two objects are in the same subset;
- **Union** merges two subsets into a single subset.

A Julia implementation of a disjoint-set is available in the `DataStructures` library. The function `DisjointSets` turns a list in a union set forest. The function `union!` will merge the sets of two elements while `in_same_set` can be used to check whether two items are in the same set. A simple example will make everything clear!

In [ ]:
using DataStructures

animals = ["mouse", "bat", "robin", "trout", "seagull", "hummingbird",
           "salmon", "goldfish", "hippopotamus", "whale", "sparrow"]
union_set_forest = DisjointSets(animals)

In [ ]:
# group mammals together
union!(union_set_forest, "mouse", "bat")
union!(union_set_forest, "mouse", "hippopotamus")
union!(union_set_forest, "whale", "bat")

In [ ]:
# group birds together
union!(union_set_forest, "robin", "seagull")
union!(union_set_forest, "seagull", "sparrow")
union!(union_set_forest, "seagull", "hummingbird")
union!(union_set_forest, "robin", "hummingbird")

In [ ]:
# group fishes together
union!(union_set_forest, "goldfish", "salmon")
union!(union_set_forest, "trout", "salmon")

In [ ]:
# mouse and whale in same subset?
in_same_set(union_set_forest, "mouse", "whale")

In [ ]:
# mouse and whale in same subset?
in_same_set(union_set_forest, "robin", "salmon")

## Heap queue

One can use a heap queue to find the minimum of a changing list without having to sort the list every update. Heaps are also implemented in `DataStructures`. The function `heapify!` will rearrange a list to satisisfy the heap property. `heappop!` and `heappush!` can be used to extract, resp. add, elements while maintaining the heap property.

In [ ]:
heap = [(5, 'A'), (3, 'B'), (2, 'C'), (7, 'D')]

heapify!(heap)  # turn in a heap

In [ ]:
# return item lowest value while retaining heap property
heappop!(heap)

This has changed the heap:

In [ ]:
heap

In [ ]:
# add new item and retain heap prop
heappush!(heap, (4, 'E'))

# Two algorithms for finding minimum spanning trees

## Prim's algorithm

Prim's algorithm starts with a single vertex and adds $|V|-1$ edges to it, always taking the next edge with a minimal weight that connects a vertex on the MST to a vertex not yet in the MST. Complete the code below.

In [ ]:
"""
    prim(vertices::Vertices{T}, edges::WeightedEdgeList{R,T},
                start::T) where {R<:Real, T}

Prim's algorithm for finding the minimum spanning tree. Inputs the vertices
(`vertices`), a list of weighted edges (`vertices`), and a starting vertex (`start`).
"""
function prim(vertices::Vertices{T}, edges::WeightedEdgeList{R,T},
                start::T) where {R<:Real, T}
    u = start
    adjlist = edges2adjlist(edges)
    ...  # complete this
    return mst_edges, cost
end

## Kruskal's algorithm


Kruskal's algorithm is a straightforward algorithm to find the minimum spanning tree. The main idea is to start with an initial 'forest' of the individual nodes of the graph. In each step of the algorithm, we add an edge with the smallest possible value that connects two disjoint trees in the forest. This process is continued until we have a single tree, which is a minimum spanning tree, or until all edges are considered. In the latter case, the algorithm returns a minimum spanning forest.

In [ ]:
"""
    kruskal(vertices::Vertices{T}, edges::WeightedEdgeList{R,T}) where {R<:Real, T}

Kruskal's algorithm for finding the minimum spanning tree. Inputs the vertices
(`vertices`) and a list of weighted edges (`vertices`).
"""
function kruskal(vertices::Vertices{T}, edges::WeightedEdgeList{R,T}) where {R<:Real, T}
    ...  # complete this
    return mst_edges, cost
end

# Ticket to ride

As an illustration, we provide the graph of the famous boardgame *Ticket To Ride* (USA version). The goal of this game is to connect two cities on a map by placing a number of trains between them. Let's load the graph!

In [ ]:
using STMO.TicketToRide

All cities included.

In [ ]:
cities

The weighted edges. The weight represents the connection cost.

In [ ]:
tickettoride_edges

Let us plot this graph. We also have the coordinates of the cities in `cities_coordinates`. It is not needed to find the MST, but can help us make a draw the map.

In [ ]:
p = plot(xaxis="longitude", yaxis="lattitude")

# add edges
for (w, c1, c2) in tickettoride_edges
    x1, y1 = cities_coordinates[c1]
    x2, y2 = cities_coordinates[c2]
    plot!(p, [x1, x2], [y1, y2], color=myred, alpha=0.8,
        lw=w, label="")
end
# plot cities
for (city, (x, y)) in cities_coordinates
    println("$city: $x, $y")
    scatter!(p, [x], [y], label="", color=mygreen, markersize=10, alpha=0.8
    , annotations=[(x, y, city)]  # comment this for clarity
    )
end
p

Your turn! Use the functions above to find a minimal spanning tree for this graph.

In [ ]:
# compute the minimum spanning tree of the ticket to ride data set
...